In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import numpy as np 
from random import shuffle
from time import time 



Using TensorFlow backend.


# 1.  Gibbs Sampling for HMMs (40 points)

## 1.1 Implementation HMM

In [2]:
train = open('en_ewt.train',encoding="utf8").read().strip()
dev = open('en_ewt.dev',encoding="utf8").read().strip()
corpus = list(map(lambda x: list(map(lambda y: y.split("\t"), x.split("\n"))),train.split("\n\n")))
corpus = corpus[:-1]

word_index = {}
index_word = {}
word_index['<s>']= 0 
word_index['</s>']= 1
index_word[0] = '<s>'
index_word[1] = '</s>'

tag_index = {}
index_tag = {}
tag_index['<s>']= 0 
tag_index['</s>']= 1
index_tag[0] = '<s>'
index_tag[1] = '</s>'

m = 2 
k = 2
for sent in corpus:
    for w, t in sent:
        if t not in tag_index:
            tag_index[t] = k 
            index_tag[k] = t
            k+= 1 
        if w not in word_index:
            word_index[w] = m 
            index_word[m] = w
            m += 1 
            
num_words = len(word_index)
e_mat = np.zeros((52,num_words))
t_mat = np.zeros((52,52))

for sent in corpus:
    prev = '<s>'
    for w, t in sent:
        i_prev,i_cur,i_word = tag_index[prev],tag_index[t],word_index[w]
        t_mat[i_prev][i_cur] +=  1
        e_mat[i_cur][i_word] += 1
        
        prev = t 
    
    t = '</s>'
    i_prev,i_cur = tag_index[prev],tag_index[t]
    t_mat[i_prev][i_cur] = t_mat[i_prev][i_cur]  + 1
    
l_t = 0.1 
l_e = 0.001

t_mat  = t_mat + l_t
e_mat = e_mat + l_e 

t_mat[0][1] = 0 
t_mat[:,0] = 0


row_sums = t_mat.sum(axis=1)
t_mat = t_mat / row_sums[:, np.newaxis]
t_mat[1] = 0

row_sums = e_mat.sum(axis=1)
e_mat = e_mat / row_sums[:, np.newaxis]
e_mat[0:2] = 0 

## 1.2 Verify implementation correctness

In [3]:
p = list(zip(t_mat[0][t_mat[0].argsort()[-5:]],t_mat[0].argsort()[-5:])) 

print("----5 tags with the highest probability of following <s>----")

for score,id_ in p[::-1]:
    print("probability: ",end="")
    print(score,end=",")
    print("tag: {}".format(index_tag[id_]))

print("----10 most probable words emitted by the adjective tag('JJ')----")

p = list(zip(e_mat[tag_index['JJ']][e_mat[tag_index['JJ']].argsort()[-10:]],
            e_mat[tag_index['JJ']].argsort()[-10:]))
for score,id_ in p[::-1]:
    print("Probability: ",end="")
    print(score,end=",")
    print("word: {}".format(index_word[id_]))


----5 tags with the highest probability of following <s>----
probability: 0.22444409022076983,tag: PRP
probability: 0.11597194548497644,tag: NNP
probability: 0.11238543078026617,tag: DT
probability: 0.07811429026859008,tag: IN
probability: 0.06568103929226109,tag: RB
----10 most probable words emitted by the adjective tag('JJ')----
Probability: 0.023070952728655104,word: other
Probability: 0.021607502232622863,word: good
Probability: 0.01678672412804607,word: new
Probability: 0.013687652489389556,word: many
Probability: 0.013687652489389556,word: great
Probability: 0.011363348760397173,word: same
Probability: 0.009555556971180875,word: sure
Probability: 0.009211215677996817,word: last
Probability: 0.008952959708108775,word: few
Probability: 0.008436447768332689,word: little


## 1.3 Implementation Gibbs

In [95]:
def preprocess(data):
    corpus = list(map(lambda x: list(map(lambda y: y.split("\t"), x.split("\n"))),data.split("\n\n")))
    sent = []
    tags = []
    for i in corpus: 
        length = 0
        ss = np.zeros(len(i))
        tt = np.zeros(len(i))
        for j in range(len(i)):
            if len(i[j]) ==  2:
                ss[j] = word_index[i[j][0]]
                tt[j] = tag_index[i[j][1]]
                length += 1
                
        if length != 0:
            sent.append(ss.astype(np.int))
            tags.append(tt.astype(np.int))
    return sent,tags

def log_prob(x,y):
    res = 0
    for i in range(len(x)-1):
        i_word,i_tag = x[i],y[i]
        next_tag = y[i+1]
        res += np.log(e_mat[i_tag][i_word]) + np.log(t_mat[i_tag][next_tag])
    res += np.log(t_mat[0][y[0]]) 
    res += np.log(t_mat[y[-1]][1]) + + np.log(e_mat[y[-1]][x[-1]])   
    return res

In [102]:
def preprocess(data):
    corpus = list(map(lambda x: list(map(lambda y: y.split("\t"), x.split("\n"))),data.split("\n\n")))
    sent = []
    tags = []
    for i in corpus: 
        length = 0
        ss = np.zeros(len(i))
        tt = np.zeros(len(i))
        for j in range(len(i)):
            if len(i[j]) ==  2:
                ss[j] = word_index[i[j][0]]
                tt[j] = tag_index[i[j][1]]
                length += 1
                
        if length != 0:
            sent.append(ss.astype(np.int))
            tags.append(tt.astype(np.int))
    return sent,tags

def run_gibbs(K,sents):
    prediction = []
    t1 = e_mat.copy()
    t2 = t_mat.copy()
    for i in range(len(sents)):
        ss = sents[i]
        leng = ss.shape[0]
        prev = np.zeros(leng+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(leng+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,leng)
        for j in range(K): 
            #changed = 0
            for l in range(leng):
                ll = l + 1 
                prob = t1[:,ss[l]] * t2[prev[l]] *t2[:,prev[ll+1]]                               
                cur[ll] =  np.random.choice(np.arange(0, 52), p=prob / prob.sum())
                #changed += (cur[ll] != prev[ll])  
            prev = cur.copy()
        prediction.append(prev[1:-1])
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += predict[i][j] == truth[i][j]
            total += 1
    return total,correct      
    
total_prob  = 0 
dev_x, dev_y = preprocess(dev)

L = [2,5,10,50,100,500,1000]
for K in L:
    a = time()
    total_prob  = 0
    prediction = run_gibbs(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    for i in range(len(dev_x)):
        total_prob += log_prob(dev_x[i],prediction[i])
    print("K = {}, ".format(K)+ "time_used:{}, accuracy: {},".format(tt,correct/total), end=" ")
    print("log prob: ", total_prob)
    

K = 2, time_used:1.2891228199005127, accuracy: 0.8201988071570576, log prob:  -177943.64827744148
K = 5, time_used:3.080998420715332, accuracy: 0.8561033797216699, log prob:  -169232.25330501987
K = 10, time_used:6.208080053329468, accuracy: 0.8631013916500994, log prob:  -168125.15533082312
K = 50, time_used:30.686946153640747, accuracy: 0.8662027833001988, log prob:  -167497.84390066314
K = 100, time_used:61.56869602203369, accuracy: 0.8669184890656063, log prob:  -167290.29541074915
K = 500, time_used:307.4152216911316, accuracy: 0.8667594433399602, log prob:  -167247.38304005214
K = 1000, time_used:613.4848682880402, accuracy: 0.8666003976143141, log prob:  -167267.10983235616


## 1.4  Sharpen  conditional distribution 

In [111]:
def run_gibbs_sharp(K,sents,b):
    prediction = []
    for i in range(len(sents)):
        ss = sents[i]
        leng = ss.shape[0]
        prev = np.zeros(ss.shape[0]+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(ss.shape[0]+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,ss.shape[0])
        for j in range(K): 
            #changed = 0
            for l in range(leng):
                ll = l + 1 
                prob = e_mat[:,ss[ll-1]] * t_mat[prev[ll-1]] *t_mat[:,prev[ll+1]]
                prob = (prob / prob.sum()) ** b
                cur[ll] = np.random.choice(np.arange(0, 52), p=prob/prob.sum())
                #changed += (cur[ll] != prev[ll])
            prev = cur.copy()
        prediction.append(prev[1:-1])
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += predict[i][j] == truth[i][j]
            total += 1
    return total,correct      
    
total_prob  = 0 
dev_x, dev_y = preprocess(dev)
B =[0.5,2,5]
L = [2,5,10,50,100,500,1000]
a = time()
for b in B:
    for K in L:
        a = time()
        total_prob  = 0
        prediction = run_gibbs_sharp(K,dev_x,b) 
        tt = time()-a    
        total,correct = evaluate(prediction,dev_y)
        for i in range(len(dev_x)):
            total_prob += log_prob(dev_x[i],prediction[i])
        print("Beta = {}, K = {}, ".format(b,K)+ "time_used: {}, accuracy: {},".format(tt,correct/total), end=" ")
        print("log prob: ", total_prob)

Beta = 0.5, K = 2, time_used: 1.4395124912261963, accuracy: 0.679324055666004, log prob:  -219693.0812928305
Beta = 0.5, K = 5, time_used: 3.5667412281036377, accuracy: 0.7565009940357853, log prob:  -195541.62069723173
Beta = 0.5, K = 10, time_used: 7.100953102111816, accuracy: 0.763976143141153, log prob:  -192966.16580132963
Beta = 0.5, K = 50, time_used: 35.52915096282959, accuracy: 0.763220675944334, log prob:  -193311.7144236441
Beta = 0.5, K = 100, time_used: 71.35182499885559, accuracy: 0.7611133200795228, log prob:  -193342.1346252105
Beta = 0.5, K = 500, time_used: 355.0567719936371, accuracy: 0.7624254473161034, log prob:  -192455.15403075365
Beta = 0.5, K = 1000, time_used: 706.8985261917114, accuracy: 0.7661630218687873, log prob:  -192376.11407479618
Beta = 2, K = 2, time_used: 1.4392297267913818, accuracy: 0.8482703777335984, log prob:  -172354.99955795892
Beta = 2, K = 5, time_used: 3.5307862758636475, accuracy: 0.8734791252485089, log prob:  -166816.05107001122
Beta = 

## 1.5 Increase beta per iteration

In [112]:
def run_gibbs_increment(K,sents):
    prediction = []
    
    for i in range(len(sents)):
        b = 0.1
        
        ss = sents[i]
        leng = ss.shape[0]
        prev = np.zeros(ss.shape[0]+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(ss.shape[0]+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,ss.shape[0])
        for j in range(K): 
            #changed = 0
            for l in range(leng):
                ll = l + 1 
                prob = e_mat[:,ss[ll-1]] * t_mat[prev[ll-1]] *t_mat[:,prev[ll+1]]
                prob = (prob / prob.sum())** b                
                cur[ll] = np.random.choice(np.arange(0, 52), p=prob / prob.sum() )
                #changed += (cur[ll] != prev[ll])
            prev = cur.copy()
            b += 0.1
        prediction.append(prev[1:-1])
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    
total_prob  = 0 
dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]
for K in L:
    a = time()
    total_prob  = 0
    prediction = run_gibbs_increment(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    for i in range(len(dev_x)):
        total_prob += log_prob(dev_x[i],prediction[i])
    print("K = {}, ".format(K)+ "time_used: {}, accuracy: {},".format(tt,correct/total), end=" ")
    print("log prob: ", total_prob)
    

K = 2, time_used: 1.5330350399017334, accuracy: 0.17924453280318092, log prob:  -418035.3159861633
K = 5, time_used: 3.7618191242218018, accuracy: 0.6948707753479125, log prob:  -214076.29194072998
K = 10, time_used: 7.486259460449219, accuracy: 0.8592047713717694, log prob:  -168781.1772020763
K = 50, time_used: 37.55096626281738, accuracy: 0.8875546719681908, log prob:  -164375.11236052364
K = 100, time_used: 75.26055526733398, accuracy: 0.8894632206759443, log prob:  -164449.1272707936
K = 500, time_used: 379.402626991272, accuracy: 0.8887475149105368, log prob:  -164299.75469682456
K = 1000, time_used: 771.6814486980438, accuracy: 0.8898608349900596, log prob:  -164339.63695466897


In [113]:
#other schedules
def run_gibbs_increment2(K,sents):
    prediction = []
    
    for i in range(len(sents)):
        b = 0.1
        ss = sents[i]
        leng = ss.shape[0]
        prev = np.zeros(ss.shape[0]+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(ss.shape[0]+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,ss.shape[0])
        for j in range(K): 
            for l in range(leng):
                ll = l + 1 
                prob = e_mat[:,ss[ll-1]] * t_mat[prev[ll-1]] *t_mat[:,prev[ll+1]]
                prob = (prob / prob.sum())** b                
                cur[ll] = np.random.choice(np.arange(0, 52), p=prob / prob.sum() )
            prev = cur.copy()
            b += 0.2
        prediction.append(prev[1:-1])
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    
total_prob  = 0 
dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]
for K in L:
    a = time()
    total_prob  = 0
    prediction = run_gibbs_increment2(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    for i in range(len(dev_x)):
        total_prob += log_prob(dev_x[i],prediction[i])
    print("K = {}, ".format(K)+ "time_used: {}, accuracy: {},".format(tt,correct/total), end=" ")
    print("log prob: ", total_prob)
    


K = 2, time_used: 1.5166070461273193, accuracy: 0.3343141153081511, log prob:  -348762.1973170238
K = 5, time_used: 3.744546890258789, accuracy: 0.8353876739562625, log prob:  -173878.89851217245
K = 10, time_used: 7.453018426895142, accuracy: 0.8758648111332008, log prob:  -166306.64404267713
K = 50, time_used: 37.49332618713379, accuracy: 0.8864811133200795, log prob:  -164720.8121806362
K = 100, time_used: 74.83993148803711, accuracy: 0.8858846918489066, log prob:  -164740.7260576843
K = 500, time_used: 387.6282739639282, accuracy: 0.8869184890656063, log prob:  -164821.6342876175
K = 1000, time_used: 784.7422511577606, accuracy: 0.8859244532803181, log prob:  -164858.43319243466


In [114]:
#other schedules
def run_gibbs_increment3(K,sents):
    prediction = []
    
    for i in range(len(sents)):
        b = 0.1
        ss = sents[i]
        leng = ss.shape[0]
        prev = np.zeros(ss.shape[0]+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(ss.shape[0]+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,ss.shape[0])
        for j in range(K): 
            for l in range(leng):
                ll = l + 1 
                prob = e_mat[:,ss[ll-1]] * t_mat[prev[ll-1]] *t_mat[:,prev[ll+1]]
                prob = (prob / prob.sum())** b                
                cur[ll] = np.random.choice(np.arange(0, 52), p=prob / prob.sum() )
            prev = cur.copy()
            b += 0.05
        prediction.append(prev[1:-1])
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    
total_prob  = 0 
dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]
for K in L:
    a = time()
    total_prob  = 0
    prediction = run_gibbs_increment3(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    for i in range(len(dev_x)):
        total_prob += log_prob(dev_x[i],prediction[i])
    print("K = {}, ".format(K)+ "time_used: {}, accuracy: {},".format(tt,correct/total), end=" ")
    print("log prob: ", total_prob)
    


K = 2, time_used: 1.5328712463378906, accuracy: 0.10751491053677932, log prob:  -451118.625712229
K = 5, time_used: 3.7557730674743652, accuracy: 0.38679920477137175, log prob:  -327846.70956859016
K = 10, time_used: 7.542876243591309, accuracy: 0.7728429423459244, log prob:  -190383.44239577083
K = 50, time_used: 37.645137786865234, accuracy: 0.8864811133200795, log prob:  -164528.5548977228
K = 100, time_used: 75.12413549423218, accuracy: 0.8902982107355865, log prob:  -164217.84898996778
K = 500, time_used: 376.1521215438843, accuracy: 0.8897415506958251, log prob:  -164192.32884740498
K = 1000, time_used: 756.204217672348, accuracy: 0.8899403578528827, log prob:  -164120.46985121048


# 2. 2. Gibbs Sampling for Minimum Bayes Risk Inference (20 points)

## 2.1 Implement MBR inference 

In [101]:
def run_mbr(K,sents):
    prediction = []
    temp_e= e_mat.copy()
    temp_t = t_mat.copy()
    
    for i in range(len(sents)):
        
        ss = sents[i]
        leng = ss.shape[0]
        accum = np.zeros((leng,52))     
        prev = np.zeros(leng+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(leng+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,leng)
        a = time()
       
        for j in range(K): 
            #changed = 0
            for l in range(leng):
                ll = l + 1 
                prob = temp_e[:,ss[l]] * temp_t[prev[l]] *temp_t[:,prev[ll+1]]
                ''' 
                prob = (prob / prob.sum()) ** b
                '''
                cur[ll] = np.random.choice(np.arange(0, 52), p=prob / prob.sum())
                accum[l][cur[ll]] += 1                 

            prev = cur.copy() 

        prediction.append(accum.argmax(axis=1))
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    


dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]
for K in L:
    a = time()
    total_prob  = 0
    prediction = run_mbr(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    for i in range(len(dev_x)):
        total_prob += log_prob(dev_x[i],prediction[i])
    print("K = {}, ".format(K)+ "time_used: {}, accuracy: {},".format(tt,correct/total), end=" ")
    print("log prob: ", total_prob)
    


K = 2, time_used: 1.360353708267212, accuracy: 0.7975745526838967, log prob:  -180285.4481064654
K = 5, time_used: 3.312302827835083, accuracy: 0.8661630218687872, log prob:  -166745.2800104561
K = 10, time_used: 6.444461345672607, accuracy: 0.879403578528827, log prob:  -164878.63967827905
K = 50, time_used: 32.03047323226929, accuracy: 0.8896222664015905, log prob:  -163837.1035614935
K = 100, time_used: 63.841551780700684, accuracy: 0.890934393638171, log prob:  -163656.04291141283
K = 500, time_used: 319.6781144142151, accuracy: 0.8930417495029821, log prob:  -163480.05759828928
K = 1000, time_used: 643.1522426605225, accuracy: 0.8935188866799205, log prob:  -163458.39784751413


## 2.2 Experiment with beta

In [108]:
def run_mbr_beta(K,sents,b):
    prediction = []
    temp_e= e_mat.copy()
    temp_t = t_mat.copy()
    
    for i in range(len(sents)):
        
        ss = sents[i]
        leng = ss.shape[0]
        accum = np.zeros((leng,52))     
        prev = np.zeros(leng+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(leng+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,leng)
        a = time()
       
        for j in range(K): 
            #changed = 0
            for l in range(leng):
                ll = l + 1 
                prob = temp_e[:,ss[l]] * temp_t[prev[l]] *temp_t[:,prev[ll+1]]      
                prob = (prob / prob.sum()) ** b
                cur[ll] =  np.random.choice(np.arange(0, 52), p=prob / prob.sum())
                accum[l][cur[ll]] += 1                 
                #changed += (cur[ll] != prev[ll])
            prev = cur.copy() 
        #print(time()-a)
        prediction.append(accum.argmax(axis=1))
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    
total_prob  = 0
t =[]
dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]
B =  [0.5,2,5]
for b in B:
    for K in L:
        a = time()
        total_prob  = 0
        prediction = run_mbr_beta(K,dev_x,b) 
        tt = time()-a    
        total,correct = evaluate(prediction,dev_y)
        for i in range(len(dev_x)):
            total_prob += log_prob(dev_x[i],prediction[i])
        print("Beta = {}, K = {},".format(b,K)+  "time_used: {}, accuracy: {}".format(tt,correct/total))


Beta = 0.5, K = 2,time_used: 1.521437168121338, accuracy: 0.6990854870775348
Beta = 0.5, K = 5,time_used: 3.7965304851531982, accuracy: 0.8295029821073558
Beta = 0.5, K = 10,time_used: 7.364254474639893, accuracy: 0.8585288270377733
Beta = 0.5, K = 50,time_used: 36.615941762924194, accuracy: 0.8842942345924454
Beta = 0.5, K = 100,time_used: 73.4118926525116, accuracy: 0.8873558648111332
Beta = 0.5, K = 500,time_used: 363.1756739616394, accuracy: 0.8911332007952286
Beta = 0.5, K = 1000,time_used: 727.5567510128021, accuracy: 0.891610337972167
Beta = 2, K = 2,time_used: 1.5297951698303223, accuracy: 0.8172564612326044
Beta = 2, K = 5,time_used: 3.6865394115448, accuracy: 0.87610337972167
Beta = 2, K = 10,time_used: 7.381760358810425, accuracy: 0.8817097415506958
Beta = 2, K = 50,time_used: 36.540788412094116, accuracy: 0.8865208747514911
Beta = 2, K = 100,time_used: 73.85675120353699, accuracy: 0.8877534791252485
Beta = 2, K = 500,time_used: 366.80619525909424, accuracy: 0.88986083499005

## 2.3. Experiment with incremental beta

In [110]:
def run_mbr_incre(K,sents):
    prediction = []
    temp_e= e_mat.copy()
    temp_t = t_mat.copy()
    
    for i in range(len(sents)):
        
        ss = sents[i]
        leng = ss.shape[0]
        accum = np.zeros((leng,52))     
        prev = np.zeros(leng+2).astype(int)
        prev[-1] = 1
        cur = np.zeros(leng+2).astype(int)
        cur[-1] = 1
        prev[1:-1] = np.random.randint(2,52,leng)
        
        b = 0.1
        for j in range(K): 
            #changed = 0            
            for l in range(leng):
                ll = l + 1 
                prob = temp_e[:,ss[l]] * temp_t[prev[l]] *temp_t[:,prev[ll+1]]
               
                prob = (prob / prob.sum()) ** b

                cur[ll] =  np.random.choice(np.arange(0, 52), p=prob / prob.sum())
                accum[l][cur[ll]] += 1                 
                #changed += (cur[ll] != prev[ll])
            prev = cur.copy() 
            b += 0.1
        
        prediction.append(accum.argmax(axis=1))
    return prediction
 
def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        for j in range(predict[i].shape[0]):
            correct += (predict[i][j] == truth[i][j])
            total += 1
    return total,correct      
    
total_prob  = 0
t =[]
dev_x, dev_y = preprocess(dev)
L = [2,5,10,50,100,500,1000]

for K in L:
    a = time()
    prediction = run_mbr_incre(K,dev_x) 
    tt = time()-a    
    total,correct = evaluate(prediction,dev_y)
    print("K = {},".format(K)+  "time_used: {}, accuracy: {}".format(tt,correct/total))
    t.append(time()-a)


K = 2,time_used: 1.5989229679107666, accuracy: 0.17041749502982106
K = 5,time_used: 3.878309965133667, accuracy: 0.6688667992047713
K = 10,time_used: 7.837512016296387, accuracy: 0.8667196819085488
K = 50,time_used: 38.78974223136902, accuracy: 0.8885884691848907
K = 100,time_used: 78.71373963356018, accuracy: 0.8900198807157058
K = 500,time_used: 393.581999540329, accuracy: 0.8894234592445328
K = 1000,time_used: 803.20441198349, accuracy: 0.8893439363817097
